## Настройка тестов

In [178]:
# Ячейка с настройками (вместо файла conftest.py)

import pytest, ipytest
ipytest.autoconfig()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

browser_name = ["chrome", "firefox"][0]   # browser selection
lang = ['en','fr','de','ru'][0]           # language selection

@pytest.fixture(scope="function")
def browser():
    if browser_name == "chrome":
        options = Options()
        options.add_experimental_option('prefs', {'intl.accept_languages': lang}) 
        browser = webdriver.Chrome(options=options)
    elif browser_name == "firefox":
        fp = webdriver.FirefoxProfile()
        fp.set_preference("intl.accept_languages", lang)
        browser = webdriver.Firefox(firefox_profile=fp)
    yield browser
    browser.quit()
    

## Локаторы

In [179]:
# Классы с переменными-локаторами (вместо файла locators.py)

from selenium.webdriver.common.by import By

class BasePageLocators():
    LOGIN_LINK = (By.CSS_SELECTOR, "#login_link")
    LOGIN_LINK_INVALID = (By.CSS_SELECTOR, "#login_link_inc")
    BASKET_LINK = (By.CSS_SELECTOR, ".basket-mini .btn")
    
class MainPageLocators():
    LOGIN_LINK = (By.CSS_SELECTOR, "#login_link")

class LoginPageLocators():
    LOGIN_FORM = (By.CSS_SELECTOR, "#login_form")
    REGISTER_FORM = (By.CSS_SELECTOR, "#register_form")

class ProductPageLocators():
    PRODUCT_NAME = (By.CSS_SELECTOR,".product_main h1")
    PRODUCT_PRICE = (By.CSS_SELECTOR,".product_main .price_color")
    ADD_BUTTON = (By.CLASS_NAME,"btn-add-to-basket")
    # после добавления в корзину
    PRODUCT_NAME_IN_BASKET = (By.CSS_SELECTOR,".alert:nth-child(1) strong")
    BASKET_VALUE = (By.CSS_SELECTOR,".alert:nth-child(3) strong")
    SUCCESS_MESSAGE = (By.CSS_SELECTOR,".alert:nth-child(1) .alertinner")
    
class BasketPageLocators():
    ITEMS = (By.CSS_SELECTOR,".basket-items")
    MESSAGE = (By.CSS_SELECTOR,"#content_inner p")
    

## Классы и методы Page Object

In [180]:
# Базовый класс (вместо файла base_page.py)

from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from math import log, sin

class BasePage():   
    def __init__(self, browser, url, timeout=10):
        self.browser = browser
        self.url = url
        self.browser.implicitly_wait(timeout)
    
    # открытие страницы
    def open(self): 
        self.browser.get(self.url)
        
    # проверка наличия элемента
    def is_element_present(self, how, what):        
        try:
            self.browser.find_element(how, what)    # how - способ поиска элемента, what - селектор
        except NoSuchElementException:
            return False
        return True
    
    # проверка отсутствия элемента 
    def is_not_element_present(self, how, what, timeout=4):  # timeout - время проверки
        try:
            WebDriverWait(self.browser, timeout).until(EC.presence_of_element_located((how, what)))
        except TimeoutException:
            return True
        return False

    # проверка исчезновения элемента
    def is_disappeared(self, how, what, timeout=4):
        try:
            WebDriverWait(self.browser, timeout, 1, TimeoutException).\
                until_not(EC.presence_of_element_located((how, what)))
        except TimeoutException:
            return False
        return True
        
    # переход на страницу логина
    def go_to_login_page(self):  
        try:
            login_link = self.browser.find_element(*BasePageLocators.LOGIN_LINK) #*:распаковка кортежа с селектором
            login_link.click()
            #alert = self.browser.switch_to.alert # если появилось всплывающее окно
            #alert.accept()
        except NoSuchElementException:
            print('\n',"Login link is not found")
    
    # переход на страницу корзины
    def go_to_basket_page(self):
        try:
            basket_link = self.browser.find_element(*BasePageLocators.BASKET_LINK) 
            basket_link.click()
        except NoSuchElementException:
            print('\n',"Basket link is not found")
        
    # проверка наличия ссылки на страницу логина
    def should_be_login_link(self):       
        assert self.is_element_present(*BasePageLocators.LOGIN_LINK), "Login link is not found"
        
    # получение проверочного кода
    def solve_quiz_and_get_code(self):     
        alert = self.browser.switch_to.alert
        x = alert.text.split(" ")[2]
        answer = str(log(abs((12 * sin(float(x))))))
        alert.send_keys(answer)
        alert.accept()
        try:
            alert = self.browser.switch_to.alert
            alert_text = alert.text
            print(f"Your code: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            print('\n',"No second alert presented")
       

### Классы страниц приложения

In [181]:
# Класс главной страницы (вместо файла main_page.py)
class MainPage(BasePage):  
    def __init__(self, *args, **kwargs):
        # заглушка (передача конструктору класса-предка аргументов класса MainPage), можно pass
        super(MainPage, self).__init__(*args, **kwargs)  

# Класс страницы логина (вместо файла login_page.py)
class LoginPage(BasePage):
    def should_be_login_page(self):
        self.should_be_login_url()
        self.should_be_login_form()
        self.should_be_register_form()
        
    # проверка корректности url адреса
    def should_be_login_url(self):  
        assert self.browser.current_url[-6:] == 'login/', 'URL is not correct'
        
    # проверка наличия формы логина
    def should_be_login_form(self):  
        assert self.is_element_present(*LoginPageLocators.LOGIN_FORM), "Login form is not found"
    
    # проверка наличия формы регистрации на странице
    def should_be_register_form(self):
        assert self.is_element_present(*LoginPageLocators.REGISTER_FORM), "Registration form is not found"

# Класс страницы товара (вместо файла product_page.py)
class ProductPage(BasePage):   
    # информация о продукте
    def product_info(self):
        try:
            product_name = self.browser.find_element(*ProductPageLocators.PRODUCT_NAME).text
            product_price = self.browser.find_element(*ProductPageLocators.PRODUCT_PRICE).text
            return product_name, product_price
        except NoSuchElementException:
            print('\n',"product name or price is not found")
    
    # добавление в корзину
    def add_to_basket(self):
        try:
            button = self.browser.find_element(*ProductPageLocators.ADD_BUTTON) 
            button.click() 
        except NoSuchElementException:
            print('\n',"ADD_BUTTON is not found")
    
    def add_to_basket_checks(self, product_info):
        self.should_be_correct_product_name(product_info[0])
        self.should_be_correct_value(product_info[1])
    
    # проверка названия в корзине
    def should_be_correct_product_name(self, product_name):
        try:
            product_name_in_basket = self.browser.find_element(*ProductPageLocators.PRODUCT_NAME_IN_BASKET).text
            assert product_name_in_basket == product_name, f"product name in the basket is '{product_name_in_basket}' \
                                                         but should be '{product_name}'"
        except NoSuchElementException:
            print('\n',"product name in the basket is not found")
              
    # проверка стоимости корзины
    def should_be_correct_value(self, product_price):
        try:
            basket_value = self.browser.find_element(*ProductPageLocators.BASKET_VALUE).text
            assert basket_value == product_price, f"basket value is '{basket_value}' \
                                                but should be '{product_price}'" 
        except NoSuchElementException:
            print('\n',"basket value is not found")
        
    
    # проверка отсутствия сообщения о добавлении в корзину
    def should_not_be_success_message(self):
        assert self.is_not_element_present(*ProductPageLocators.SUCCESS_MESSAGE), \
           "Success message is presented, but should not be"
    
    # проверка исчезновения сообщения о добавлении в корзину
    def should_disappear_success_message(self):
        assert self.is_disappeared(*ProductPageLocators.SUCCESS_MESSAGE), \
           "Success message is still presented, but should disappear"

class BasketPage(BasePage):
    # проверка отсутствия предметов в корзине
    def should_not_be_products(self):
        assert self.is_not_element_present(*BasketPageLocators.ITEMS), \
           "items are presented, but should not be"
    
    # проверка наличия сообщения о том, что корзина пуста
    def should_be_correct_message(self):
        try:
            message = self.browser.find_element(*BasketPageLocators.MESSAGE).text
            correct_message = "Your basket is empty. Continue shopping"
            assert message == correct_message, f"message is '{message}' but should be '{correct_message}'"
        except NoSuchElementException:
            print('\n',"Message is not found")
              

## Тест-кейсы

### - главной страницы (вместо файла test_main_page.py)

In [189]:
%%ipytest -qq --tb=line 

link = "http://selenium1py.pythonanywhere.com/"

# TC1. Проверка наличия ссылки на страницу логина
def test_guest_should_see_login_link(browser):
    page = MainPage(browser, link)
    page.open()
    page.should_be_login_link()
    
# TC2. Проверка перехода на страницу логина
def test_guest_can_go_to_login_page(browser):  
    page = MainPage(browser, link)   # инициализируем Page Object, передаем в конструктор экземпляр драйвера и url 
    page.open()                      # открываем страницу
    page.go_to_login_page()          # выполняем метод страницы — переходим на страницу логина
    login_page = LoginPage(browser, browser.current_url)
    login_page.should_be_login_page()

# TC3. Проверка отсутствия товаров в корзине
def test_guest_cant_see_product_in_basket_opened_from_main_page(browser):
    page = MainPage(browser, link)
    page.open()
    page.go_to_basket_page()          
    basket_page = BasketPage(browser, browser.current_url)
    basket_page.should_not_be_products()
    basket_page.should_be_correct_message()

...                                                                                          [100%]


### - страниц товара (вместо файла test_product_page.py)

In [183]:
%%ipytest -qq -s --tb=line

'''Задание: добавление в корзину со страницы товара
1. Открываем страницу товара (http://selenium1py.pythonanywhere.com/catalogue/the-shellcoders-handbook_209/?promo=newYear).
Обратите внимание, что в ссылке есть параметр "?promo=newYear". Не теряйте его в авто-тесте, чтобы получить проверочный код.
2. Нажимаем на кнопку "Добавить в корзину".
3. *Посчитать результат математического выражения и ввести ответ. Используйте для этого метод solve_quiz_and_get_code(),
который приведен ниже. Например, можете добавить его в класс BasePage, чтобы использовать его на любой странице. 
Этот метод нужен только для проверки того, что вы написали тест на Selenium. После этого вы получите код, который нужно 
ввести в качестве ответа на данное задание. Код будет выведен в консоли интерпретатора, в котором вы запускаете тест. 
Не забудьте в конце теста добавить проверки на ожидаемый результат.

Ожидаемый результат: 
1. Сообщение о том, что товар добавлен в корзину. Название товара в сообщении должно совпадать с тем товаром,
который вы действительно добавили.
2. Сообщение со стоимостью корзины. Стоимость корзины совпадает с ценой товара. '''

#link = 'http://selenium1py.pythonanywhere.com/catalogue/the-shellcoders-handbook_209/?promo=newYear'
link = 'http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=newYear2019'

# TC4. Добавление в корзину со страницы товара
def test_guest_can_add_product_to_basket(browser):
    page = ProductPage(browser, link)
    page.open()
    product_info = page.product_info()
    page.add_to_basket()
    page.solve_quiz_and_get_code()
    page.add_to_basket_checks(product_info)
    

Your code: Поздравляем, вы справились! Вставьте это число в поле ответа на Stepik: 27.258362256656856
.


In [184]:
%%ipytest -qq -s --tb=line

'''Задание: независимость контента, ищем баг
Для нашего интернет-магазина было запущено несколько новых промо-акций, одна из которых привела к появлению бага.
Промо-акция включается путем добавления параметра ?promo=offerN к ссылке на товар.
К счастью, нам не придется менять наш тест, чтобы проверить изменения в коде. Мы просто запустим всё тот же тест на
странице http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/ с параметризацией. Вам нужно определить,
при каком значении параметра promo автотест упадет. Для этого проверьте результат работы PyTest и найдите url, на котором
произошла ошибка. Значение параметра может изменяться от offer0 до offer9.
После того как вы обнаружили баг, учитывая что чинить его не собираются, лучше всего пометить падающий тест как xfail
или skip.'''

s = 'http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/'

# TC5. Добавление в корзину со страницы товара при новых промо-акциях
@pytest.mark.parametrize('n', [*range(7), pytest.param(7, marks=pytest.mark.xfail(reason='bugged')), *range(8, 10)])
def test_guest_can_add_product_to_basket(browser, n):
    link = s + '?promo=offer' + str(n)
    page = ProductPage(browser, link)
    page.open()
    product_info = page.product_info()
    page.add_to_basket()
    page.solve_quiz_and_get_code()
    page.add_to_basket_checks(product_info)
    


 No second alert presented
.
 No second alert presented
.
 No second alert presented
.
 No second alert presented
.
 No second alert presented
.
 No second alert presented
.
 No second alert presented
.
 No second alert presented
x
 No second alert presented
.
 No second alert presented
.


In [185]:
%%ipytest -qq -s --tb=line

'''Задание: отрицательные проверки
Добавьте к себе в проект функции с отрицательными проверками и реализуйте несколько простых тестов: 

test_guest_cant_see_success_message_after_adding_product_to_basket: 
Открываем страницу товара 
Добавляем товар в корзину 
Проверяем, что нет сообщения об успехе с помощью is_not_element_present
 
test_guest_cant_see_success_message: 
Открываем страницу товара 
Проверяем, что нет сообщения об успехе с помощью is_not_element_present
 
test_message_disappeared_after_adding_product_to_basket: 
Открываем страницу товара
Добавляем товар в корзину
Проверяем, что нет сообщения об успехе с помощью is_disappeared
 
Запустите все три теста. Те тесты, которые упали, пометьте как XFail или skip'''

link = "https://selenium1py.pythonanywhere.com/en-gb/catalogue/coders-at-work_207/"

# TC6
@pytest.mark.xfail(reason='incorrect test')
def test_guest_cant_see_success_message_after_adding_product_to_basket(browser):
    page = ProductPage(browser, link)
    page.open() 
    page.add_to_basket()
    page.should_not_be_success_message()

# TC7
def test_guest_cant_see_success_message(browser):
    page = ProductPage(browser, link)
    page.open() 
    page.should_not_be_success_message()
    
# TC8
@pytest.mark.xfail(reason='the feature is not ready')
def test_message_disappeared_after_adding_product_to_basket(browser):
    page = ProductPage(browser, link)
    page.open() 
    page.add_to_basket()
    page.should_disappear_success_message()
    

x.x


In [186]:
%%ipytest -qq -s --tb=line

link = "http://selenium1py.pythonanywhere.com/en-gb/catalogue/the-city-and-the-stars_95/"

# TC9. Проверка наличия ссылки на страницу логина со страницы товара
def test_guest_should_see_login_link_on_product_page(browser):  
    page = ProductPage(browser, link)
    page.open()
    page.should_be_login_link()
    
# TC10. Проверка перехода на страницу логина со страницы товара
def test_guest_can_go_to_login_page_from_product_page(browser):
    page = ProductPage(browser, link)    
    page.open()                      
    page.go_to_login_page()          
    login_page = LoginPage(browser, browser.current_url)
    login_page.should_be_login_page()
    

..


In [187]:
%%ipytest -qq -s --tb=line

'''Задание: наследование и отрицательные проверки
В файл test_main_page.py добавьте тест с названием test_guest_cant_see_product_in_basket_opened_from_main_page:
Гость открывает главную страницу 
Переходит в корзину по кнопке в шапке сайта
Ожидаем, что в корзине нет товаров
Ожидаем, что есть текст о том что корзина пуста 

В файле test_product_page.py добавьте тест с названием test_guest_cant_see_product_in_basket_opened_from_product_page:
Гость открывает страницу товара
Переходит в корзину по кнопке в шапке 
Ожидаем, что в корзине нет товаров
Ожидаем, что есть текст о том что корзина пуста 

В классе BasePage реализуйте соответствующий метод для перехода в корзину. Создайте файл basket_page.py и в нем класс
BasketPage. Реализуйте там необходимые проверки, в том числе отрицательную, которую мы обсуждали в предыдущих шагах.

NB: в этой ячейке реализован 2-й тест. Первый - в тестах главной страницы'''

# TC11. Проверка отсутствия товаров в корзине
def test_guest_cant_see_product_in_basket_opened_from_product_page(browser):
    page = ProductPage(browser, link)
    page.open()
    page.go_to_basket_page()          
    basket_page = BasketPage(browser, browser.current_url)
    basket_page.should_not_be_products()
    basket_page.should_be_correct_message()

.
